In [6]:
import pandas as pd

In [7]:
origin = pd.read_json('https://www.wikiart.org/en/App/Painting/MostViewedPaintings?randomSeed=123&json=2')
origin.head(3)

,artistContentId,artistName,completitionYear,contentId,height,image,title,width,yearAsString
0,225091,Leonardo da Vinci,1504.0,225189,4289,https://uploads7.wikiart.org/images/leonardo-d...,Mona Lisa,2835,1504.0
1,204915,Vincent van Gogh,1889.0,207190,1600,https://uploads3.wikiart.org/images/vincent-va...,The Starry Night,2560,1889.0
2,220740,Salvador Dali,1931.0,221654,1600,https://uploads6.wikiart.org/images/salvador-d...,The Persistence of Memory,2105,1931.0


In [8]:
print('Total images:', len(origin.index))
filtered = origin
filtered = filtered[filtered.image.str.contains('FRAME') == False]
print('After filtering empty frames:', len(filtered.index))
filtered = filtered[filtered.width >= filtered.height]
print('After filtering vertical images:', len(filtered.index))
filtered = filtered[filtered.width >= 1000]
print('After filtering small:', len(filtered.index))

('Total images:', 600)
('After filtering empty frames:', 600)
('After filtering vertical images:', 292)
('After filtering small:', 233)


In [9]:
OUT_FILE = '../server/data/images.csv'
filtered.image = filtered.image.str.replace('!Large.jpg', '')
filtered.to_csv(OUT_FILE, sep=',', encoding='utf-8', index=False)